In [11]:
import numpy as np
import numpy.linalg as npl
import scipy as sp
import scipy.linalg as spl
import scipy.sparse as sps
import matplotlib.pyplot as plt
%matplotlib widget

pi = np.pi

# Exercice 1 : Equation de transport

$$u^{n+1}_{j} = u^{n-1}_j + c \frac{\partial t}{\partial x} (u^n_{j+1} - u^n_{j-1})$$

In [2]:
def u0(x, mu=0, sigma2=1):
    exp = np.exp(-0.5 * ((x-mu)**2)/sigma2)
    return exp/np.sqrt(2*pi*sigma2)

In [3]:
def v0(x, k=1, mu=0, sigma2=1):
    return u0(x, mu, sigma2) * np.sin(2*k*pi*x)

In [4]:
def w0(x, a=-2.5, b=2.5):
    return np.where((x>=a) & (x<=b), 1, 0)

In [5]:
def sch_Saute_Mouton(U0, U1, c, dt, dx, T):
    U = [U0, U1]
    N = U0.size
    A = c*dt/dx * sps.diags([1, -1, 1, -1], [-N, -1, 1, N], shape=[N, N]).toarray() 
    
    n = 2
    
    while n*dt <= T:
        U2 = U[-2] - A.dot(U[-1])
        U.append(U2)
        n += 1
    
    return U

In [14]:
def sch_décentré_amont(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) - c*dt/(dx) * sps.diags([-1, 1, -1], [-1, 0, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [27]:
def test_sch_décentré_amont(min, max, c, dt=0.001, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré_amont(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [30]:
test_sch_décentré_amont(-5, 5, 0.5, dt=0.24, T=4)

/home/pjbenard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/pjbenard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/pjbenard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def sch_décentré(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) + c*dt/dx * sps.diags([-0.5, 1.5, -1, -0.5, 1.5], [-N+1, -1, 0, 1, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [7]:
def test_sch_décentré(min, max, c, dt=0.005, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [12]:
test_sch_décentré(-5, 5, 0.5, T=4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …